In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import json
from pathlib import Path

cfg_path = Path("/content/drive/MyDrive/UrbanSimAI_Chicago/urbansim_config.json")
cfg = json.loads(cfg_path.read_text())

PROJECT_ROOT = Path(cfg["PROJECT_ROOT"])
RAW_DIR = Path(cfg["RAW_DIR"])
PROC_DIR = Path(cfg["PROC_DIR"])
REPORT_DIR = Path(cfg["REPORT_DIR"])

print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR:", RAW_DIR)
print("PROC_DIR:", PROC_DIR)
print("REPORT_DIR:", REPORT_DIR)

PROC_DIR.mkdir(parents=True, exist_ok=True)
REPORT_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_ROOT: /content/drive/MyDrive/UrbanSimAI_Chicago
RAW_DIR: /content/drive/MyDrive/UrbanSimAI_Chicago/data_raw
PROC_DIR: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed
REPORT_DIR: /content/drive/MyDrive/UrbanSimAI_Chicago/reports


In [ ]:
!pip -q install duckdb rtree pyarrow fastparquet geopandas shapely pyogrio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00


In [ ]:
import os, time
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timezone, timedelta
from pathlib import Path

import geopandas as gpd
from shapely.geometry import Point, LineString

import duckdb

def stamp():
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

def file_gb(p: Path) -> float:
    return (p.stat().st_size / (1024**3)) if p.exists() else 0.0

def folder_gb(folder: Path) -> float:
    total = 0
    if folder.exists():
        for p in folder.rglob("*"):
            if p.is_file():
                total += p.stat().st_size
    return total / (1024**3)

def find_col(cols, candidates):
    low = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand.lower() in low:
            return low[cand.lower()]
    # contains fallback
    for c in cols:
        for cand in candidates:
            if cand.lower() in c.lower():
                return c
    return None

print("Helpers ready:", stamp())

Helpers ready: 2025-12-14 04:14:53 UTC


In [ ]:
zip_dir = RAW_DIR / "boundaries_zip"
zip_dir.mkdir(parents=True, exist_ok=True)
zip_geojson = zip_dir / "chicago_zip_boundaries.geojson"

ZIP_VIEW_CANDIDATES = ["unjd-c2ca", "gdcf-axmw"]

def download_socrata_geojson(view_id: str, out_path: Path):
    url = f"https://data.cityofchicago.org/resource/{view_id}.geojson?$limit=50000"
    r = requests.get(url, timeout=(30, 2400))
    if r.status_code != 200:
        raise RuntimeError(f"{view_id} failed: {r.status_code}")
    out_path.write_bytes(r.content)

if not zip_geojson.exists():
    ok = False
    for vid in ZIP_VIEW_CANDIDATES:
        try:
            print("⬇️ Trying ZIP boundaries:", vid)
            download_socrata_geojson(vid, zip_geojson)
            print("✅ Saved:", zip_geojson)
            ok = True
            break
        except Exception as e:
            print("⚠️ Failed:", vid, "|", e)
    if not ok:
        raise RuntimeError("Could not download ZIP boundaries. Try again later.")
else:
    print("⏭️ Skip (exists):", zip_geojson)

⬇️ Trying ZIP boundaries: unjd-c2ca
✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_raw/boundaries_zip/chicago_zip_boundaries.geojson


In [ ]:
zip_gdf = gpd.read_file(zip_geojson)

if zip_gdf.crs is None:
    zip_gdf = zip_gdf.set_crs(epsg=4326)
else:
    zip_gdf = zip_gdf.to_crs(epsg=4326)

zip_col = find_col(zip_gdf.columns.tolist(), ["zip", "zip_code", "zipcode", "zipcodes", "zip codes"])
if zip_col is None:
    print("Columns:", zip_gdf.columns.tolist())
    raise ValueError("Could not find ZIP column in boundaries file.")

zip_gdf = zip_gdf[[zip_col, "geometry"]].rename(columns={zip_col: "zip"})
zip_gdf["zip"] = zip_gdf["zip"].astype(str).str.zfill(5)

print("✅ ZIP polygons:", len(zip_gdf), "| sample:", zip_gdf["zip"].head().tolist())

zip_out = PROC_DIR / "zip_boundaries.parquet"
zip_gdf.to_parquet(zip_out, index=False)
print("✅ Saved:", zip_out)

✅ ZIP polygons: 59 | sample: ['60607', '60661', '60606', '60615', '60643']
✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/zip_boundaries.parquet


In [ ]:
traffic_dir = RAW_DIR / "chicago_traffic"
latest_csv = traffic_dir / "traffic_latest_n4j6_wkkf.csv"
assert latest_csv.exists(), f"Missing: {latest_csv}"

df_latest = pd.read_csv(latest_csv)
cols = df_latest.columns.tolist()

seg_col = find_col(cols, ["segmentid", "segment_id", "SEGMENTID"])
slon = find_col(cols, ["start_longitude", "START_LONGITUDE"])
slat = find_col(cols, ["start_latitude", "START_LATITUDE"])
elon = find_col(cols, ["end_longitude", "END_LONGITUDE"])
elat = find_col(cols, ["end_latitude", "END_LATITUDE"])

need = [seg_col, slon, slat, elon, elat]
if any(c is None for c in need):
    print("Columns:", cols)
    raise ValueError("Missing needed coordinate columns in latest traffic snapshot.")

seg = df_latest[[seg_col, slon, slat, elon, elat]].copy()
seg = seg.rename(columns={seg_col:"segment_id", slon:"slon", slat:"slat", elon:"elon", elat:"elat"})
seg = seg.dropna()
seg["segment_id"] = pd.to_numeric(seg["segment_id"], errors="coerce")
seg = seg.dropna(subset=["segment_id"]).copy()
seg["segment_id"] = seg["segment_id"].astype(int)

# geometry
seg["geometry"] = seg.apply(lambda r: LineString([(r["slon"], r["slat"]), (r["elon"], r["elat"])]), axis=1)
seg["midpoint"] = seg.apply(lambda r: Point((r["slon"]+r["elon"])/2, (r["slat"]+r["elat"])/2), axis=1)

seg_gdf = gpd.GeoDataFrame(seg[["segment_id", "geometry"]], geometry="geometry", crs="EPSG:4326")
mid_gdf = gpd.GeoDataFrame(seg[["segment_id"]].copy(), geometry=seg["midpoint"], crs="EPSG:4326")

print("✅ Segments built:", len(seg_gdf))

✅ Segments built: 1257


In [ ]:
# join midpoints into ZIP polygons
joined = gpd.sjoin(mid_gdf, zip_gdf, how="left", predicate="within")
seg2zip = joined[["segment_id", "zip"]].drop_duplicates()

missing = seg2zip["zip"].isna().mean()
print("✅ segment→zip rows:", len(seg2zip), "| missing zip %:", round(missing*100, 2))

seg2zip_out = PROC_DIR / "segment_to_zip.parquet"
seg2zip.to_parquet(seg2zip_out, index=False)
print("✅ Saved:", seg2zip_out)

✅ segment→zip rows: 1257 | missing zip %: 12.65
✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/segment_to_zip.parquet


In [ ]:
big_csv = traffic_dir / "traffic_hist_2024_current_4g9f_3jbs.csv"
assert big_csv.exists(), f"Missing: {big_csv}"

# Change this if you want more/less:
LAST_DAYS = 90
start_ts = (datetime.now(timezone.utc) - timedelta(days=LAST_DAYS)).strftime("%Y-%m-%d %H:%M:%S")
print("Using LAST_DAYS =", LAST_DAYS, "| start_ts =", start_ts)

# read header only to detect columns
cols_4g9f = pd.read_csv(big_csv, nrows=0).columns.tolist()

seg_col = find_col(cols_4g9f, ["segmentid", "segment_id", "SEGMENTID"])
time_col = find_col(cols_4g9f, ["time", "measurement_start", "measurement_time", "date_time", "timestamp"])
speed_col = find_col(cols_4g9f, ["speed", "avg_speed", "current_speed"])
travel_col = find_col(cols_4g9f, ["travel_time", "traveltime", "trip_time"])

print("Detected:", {"segment":seg_col, "time":time_col, "speed":speed_col, "travel_time":travel_col})

if seg_col is None or time_col is None:
    raise ValueError("Could not detect segment/time columns in 4g9f CSV. Print columns and adjust candidates.")

out_4g9f = PROC_DIR / f"traffic_4g9f_last{LAST_DAYS}d.parquet"
if out_4g9f.exists():
    print("⏭️ Skip (exists):", out_4g9f)
else:
    con = duckdb.connect(database=":memory:")
    con.execute("PRAGMA threads=4;")

    # Build SELECT list dynamically (only include columns that exist)
    select_exprs = [
        f'CAST("{seg_col}" AS BIGINT) AS segment_id',
        f'CAST("{time_col}" AS TIMESTAMP) AS ts'
    ]
    if speed_col:
        select_exprs.append(f'CAST("{speed_col}" AS DOUBLE) AS speed')
    if travel_col:
        select_exprs.append(f'CAST("{travel_col}" AS DOUBLE) AS travel_time')

    select_sql = ",\n        ".join(select_exprs)

    # join zip mapping inside duckdb
    con.execute(f"""
        COPY (
            SELECT
                t.*,
                m.zip
            FROM (
                SELECT
                    {select_sql}
                FROM read_csv_auto('{big_csv.as_posix()}', header=True, ignore_errors=True)
                WHERE CAST("{time_col}" AS TIMESTAMP) >= TIMESTAMP '{start_ts}'
            ) t
            LEFT JOIN read_parquet('{seg2zip_out.as_posix()}') m
            ON t.segment_id = m.segment_id
        )
        TO '{out_4g9f.as_posix()}'
        (FORMAT PARQUET, COMPRESSION 'snappy');
    """)
    print("✅ Saved:", out_4g9f, "| GB:", round(file_gb(out_4g9f), 3))

Using LAST_DAYS = 90 | start_ts = 2025-09-15 04:18:10
Detected: {'segment': 'SEGMENT_ID', 'time': 'TIME', 'speed': 'SPEED', 'travel_time': None}


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/traffic_4g9f_last90d.parquet | GB: 0.027


In [ ]:
sxs8_parts = traffic_dir / "sxs8_h27x_mini_parts"

out_sxs8 = PROC_DIR / "traffic_sxs8_mini.parquet"
if sxs8_parts.exists():
    if out_sxs8.exists():
        print("⏭️ Skip (exists):", out_sxs8)
    else:
        con = duckdb.connect(database=":memory:")
        con.execute("PRAGMA threads=4;")

        # read all parts as one dataset
        con.execute(f"""
            COPY (
                SELECT p.*, m.zip
                FROM read_parquet('{(sxs8_parts/"*.parquet").as_posix()}') p
                LEFT JOIN read_parquet('{seg2zip_out.as_posix()}') m
                ON CAST(p.segment_id AS BIGINT) = m.segment_id
            )
            TO '{out_sxs8.as_posix()}'
            (FORMAT PARQUET, COMPRESSION 'snappy');
        """)
        print("✅ Saved:", out_sxs8, "| GB:", round(file_gb(out_sxs8), 3))
else:
    print("ℹ️ sxs8 mini parts not found, skipping.")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/traffic_sxs8_mini.parquet | GB: 0.046


In [ ]:
zip_hourly = PROC_DIR / "traffic_zip_hourly.parquet"
if zip_hourly.exists():
    print("⏭️ Skip (exists):", zip_hourly)
else:
    con = duckdb.connect(database=":memory:")
    con.execute("PRAGMA threads=4;")

    con.execute(f"""
        COPY (
            SELECT
                zip,
                date_trunc('hour', ts) AS hour_ts,
                avg(speed) AS avg_speed,
                count(*) AS n_obs
            FROM read_parquet('{out_4g9f.as_posix()}')
            WHERE zip IS NOT NULL AND speed IS NOT NULL
            GROUP BY 1,2
        )
        TO '{zip_hourly.as_posix()}'
        (FORMAT PARQUET, COMPRESSION 'snappy');
    """)
    print("✅ Saved:", zip_hourly, "| GB:", round(file_gb(zip_hourly), 3))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/traffic_zip_hourly.parquet | GB: 0.001


In [ ]:
zillow_csv = RAW_DIR / "zillow" / "Zip_ZHVI.csv"
assert zillow_csv.exists(), f"Missing: {zillow_csv}"

# Chicago ZIP list from boundaries
chicago_zips = set(zip_gdf["zip"].unique().tolist())

z = pd.read_csv(zillow_csv)
# RegionName is ZIP in Zillow file
if "RegionName" not in z.columns:
    raise ValueError("Zillow file: RegionName column not found.")

z["RegionName"] = z["RegionName"].astype(str).str.zfill(5)
z_chi = z[z["RegionName"].isin(chicago_zips)].copy()

# keep only date columns + ZIP
date_cols = [c for c in z_chi.columns if c[:4].isdigit()]  # "YYYY-MM-DD"
keep = ["RegionName"] + date_cols
z_chi = z_chi[keep].rename(columns={"RegionName": "zip"})

# wide -> long
z_long = z_chi.melt(id_vars=["zip"], var_name="date", value_name="zhvi")
z_long["date"] = pd.to_datetime(z_long["date"], errors="coerce")
z_long = z_long.dropna(subset=["date"])

z_out = PROC_DIR / "zillow_zhvi_chicago_zip_long.parquet"
z_long.to_parquet(z_out, index=False)
print("✅ Zillow Chicago ZIP rows:", len(z_long), "| Saved:", z_out)

✅ Zillow Chicago ZIP rows: 17980 | Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/zillow_zhvi_chicago_zip_long.parquet


In [ ]:
gtfs_dir = RAW_DIR / "cta_gtfs" / "extracted"
stops_txt = gtfs_dir / "stops.txt"
assert stops_txt.exists(), f"Missing: {stops_txt}"

stops = pd.read_csv(stops_txt)
need = ["stop_lat", "stop_lon"]
if any(c not in stops.columns for c in need):
    raise ValueError("GTFS stops.txt missing stop_lat/stop_lon")

stops_gdf = gpd.GeoDataFrame(
    stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]].copy(),
    geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
    crs="EPSG:4326"
)

stops_join = gpd.sjoin(stops_gdf, zip_gdf, how="left", predicate="within")
stops_zip = stops_join.groupby("zip").size().reset_index(name="cta_stop_count")

stops_out = PROC_DIR / "cta_stops_by_zip.parquet"
stops_zip.to_parquet(stops_out, index=False)

print("✅ CTA stops by zip:", len(stops_zip), "| Saved:", stops_out)

✅ CTA stops by zip: 59 | Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/data_processed/cta_stops_by_zip.parquet


In [ ]:
manifest2 = {
    "processed_at": stamp(),
    "files": {
        "zip_boundaries": str(zip_out),
        "segment_to_zip": str(seg2zip_out),
        "traffic_4g9f_recent": str(out_4g9f),
        "traffic_zip_hourly": str(zip_hourly),
        "zillow_zhvi_chicago_long": str(z_out),
        "cta_stops_by_zip": str(stops_out),
    }
}

manifest_path = REPORT_DIR / "processed_manifest.json"
manifest_path.write_text(json.dumps(manifest2, indent=2))
print("✅ Saved:", manifest_path)

print("\n--- PROC_DIR size check (top files) ---")
all_files = []
for p in PROC_DIR.rglob("*"):
    if p.is_file():
        all_files.append((p, p.stat().st_size))
all_files.sort(key=lambda x: x[1], reverse=True)
for p, sz in all_files[:10]:
    print(f"{p.relative_to(PROJECT_ROOT)} | {sz/1e6:.2f} MB")

print("\n--- Preview: traffic_zip_hourly ---")
print(pd.read_parquet(zip_hourly).head())

✅ Saved: /content/drive/MyDrive/UrbanSimAI_Chicago/reports/processed_manifest.json

--- PROC_DIR size check (top files) ---
data_processed/traffic_sxs8_mini.parquet | 49.16 MB
data_processed/traffic_4g9f_last90d.parquet | 28.77 MB
data_processed/traffic_zip_hourly.parquet | 0.76 MB
data_processed/zip_boundaries.parquet | 0.55 MB
data_processed/zillow_zhvi_chicago_zip_long.parquet | 0.18 MB
data_processed/segment_to_zip.parquet | 0.01 MB
data_processed/cta_stops_by_zip.parquet | 0.00 MB

--- Preview: traffic_zip_hourly ---
     zip             hour_ts  avg_speed  n_obs
0  60661 2025-09-15 05:00:00  14.740000     50
1  60642 2025-09-15 05:00:00  13.552941     85
2  60605 2025-09-15 05:00:00  17.060870    115
3  60603 2025-09-15 05:00:00  19.066667     15
4  60638 2025-09-15 05:00:00  20.348148    135


In [ ]:
#END